In [ ]:
# Add path
import sys
sys.path.insert(1, '../src')

from BitFlipper import add_BitFlips
from MeasureMatrix import fast_interaction_multiply
from Qubit_Simulator import pad_counts
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
import random
from tqdm import tqdm

def CZ_errors(num_gates):
    qc = QuantumCircuit(2)

    for i in range(num_gates):
        qc.cz(0,1)
    qc.measure_all()

    return qc

In [ ]:
num_qubits = 2
sim_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
shots_per_circuit = 8192
e_g1 = 0.0035
e_g2 = 0.0027
num_gates = 0 # round(0.2/(max(e_g1,e_g2)))
test_circuit = CZ_errors(num_gates)

simulator = AerSimulator()
flipped_counts = np.zeros(4)
bitstring_list = [
        format(i, f'0{num_qubits}b') 
        for i in range(2**num_qubits)
    ]

for sim_seed in tqdm(sim_seeds):
    flipped_circuit = add_BitFlips(test_circuit, seed = sim_seed, gate_errors = [0,0])
    flipped_result = simulator.run(flipped_circuit, shots=shots_per_circuit).result()
    counts = pad_counts(flipped_result.get_counts(),2)
    bitstring_counts = [counts.get(key, 0) for key in bitstring_list]
    flipped_counts += bitstring_counts

print(bitstring_list)

flipped_counts = fast_interaction_multiply(flipped_counts,num_qubits,[0.0035,0.0027],[0.0085,0.0072])

raw_output = []

for i, bitstr in enumerate(bitstring_list):
    raw_output.extend([bitstr] * round(flipped_counts[i]))

if len(raw_output) > len(sim_seeds)*shots_per_circuit:
    raw_output = raw_output[:len(sim_seeds)*shots_per_circuit]
elif len(raw_output) < len(sim_seeds)*shots_per_circuit:
    diff = len(sim_seeds)*shots_per_circuit - len(raw_output)
    raw_output.extend([raw_output[0]] * diff)

print(len(raw_output))

raw_output = np.random.permutation(raw_output)

np.savetxt('../data/tes_data.csv', np.array(raw_output).reshape(1, -1), delimiter=',', fmt='%s')